## Web Scraping usando Beautiful Soup

Si es que no tienen instalado Beautiful Soup, lo instalamos con el siguiente comando:

In [11]:
#!pip install beautifulsoup4

In [2]:
# Importamos nuestros paquetes
import pandas as pd
import numpy as np
import pprint
import json
import requests
from bs4 import BeautifulSoup as bs

In [3]:
pagina = requests.get("http://www.alrb.org/datacleaning/covidcaseoutliers.html")

In [6]:
sopa = bs(pagina.text,'html.parser')

In [10]:
#print(sopa)

In [7]:
cabeceras = sopa.find('table', {'id':'tblDeaths'}).thead.find_all('th')

In [8]:
type(cabeceras)

bs4.element.ResultSet

In [12]:
nombresCols= [j.get_text() for j in cabeceras]

In [13]:
nombresCols

['Country',
 'Cases',
 'Deaths',
 'Cases per Million',
 'Deaths per Million',
 'population',
 'population_density',
 'median_age',
 'gdp_per_capita',
 'hospital_beds_per_100k']

In [14]:
filas = sopa.find('table',{'id':'tblDeaths'}).tbody.find_all('tr')

In [20]:
#definimos un lista para los datos de la fila y un lista para los datos de labeling
datosFila=[]
labelFila=[]

for fila in filas:
    label = fila.find('th').get_text()
    celdas = fila.find_all('td', {'class': 'data'})
    labelFila.append(label)
    #print(label)
    textoCelda = [i.get_text() for i in celdas ]
    datosFila.append(textoCelda)
    


Tenemos que agregar a datosFila, labelFila

In [27]:
for i in range(len(datosFila)):
    datosFila[i].insert(0,labelFila[i])

Crearemos nuestro objeto `pandas.DataFrame`

In [28]:
totalMuertes = pd.DataFrame(datosFila,columns=nombresCols)

In [29]:
totalMuertes.head()

,Country,Cases,Deaths,Cases per Million,Deaths per Million,population,population_density,median_age,gdp_per_capita,hospital_beds_per_100k
0,Algeria,"9,394",653,214,15,"43,851,043",17,29,"13,914",1.9
1,Austria,"16,642",668,1848,74,"9,006,400",107,44,"45,437",7.4
2,Bangladesh,"47,153",650,286,4,"164,689,383",1265,28,"3,524",0.8
3,Belgium,"58,381",9467,5037,817,"11,589,616",376,42,"42,659",5.6
4,Brazil,"514,849",29314,2422,138,"212,559,409",25,34,"14,103",2.2


In [30]:
totalMuertes.dtypes

Country                   object
Cases                     object
Deaths                    object
Cases per Million         object
Deaths per Million        object
population                object
population_density        object
median_age                object
gdp_per_capita            object
hospital_beds_per_100k    object
dtype: object

In [31]:
for columna in totalMuertes.columns[1:-1]:
    totalMuertes[columna] = totalMuertes[columna].str.replace("[^0-9]","").astype("int64")

<ipython-input-31-84bfb34683dc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  totalMuertes[columna] = totalMuertes[columna].str.replace("[^0-9]","").astype("int64")


In [32]:
totalMuertes.dtypes

Country                   object
Cases                      int64
Deaths                     int64
Cases per Million          int64
Deaths per Million         int64
population                 int64
population_density         int64
median_age                 int64
gdp_per_capita             int64
hospital_beds_per_100k    object
dtype: object

In [33]:
totalMuertes['hospital_beds_per_100k'] = totalMuertes['hospital_beds_per_100k'].astype("float")

In [34]:
totalMuertes.dtypes

Country                    object
Cases                       int64
Deaths                      int64
Cases per Million           int64
Deaths per Million          int64
population                  int64
population_density          int64
median_age                  int64
gdp_per_capita              int64
hospital_beds_per_100k    float64
dtype: object

In [35]:
totalMuertes.head()

,Country,Cases,Deaths,Cases per Million,Deaths per Million,population,population_density,median_age,gdp_per_capita,hospital_beds_per_100k
0,Algeria,9394,653,214,15,43851043,17,29,13914,1.9
1,Austria,16642,668,1848,74,9006400,107,44,45437,7.4
2,Bangladesh,47153,650,286,4,164689383,1265,28,3524,0.8
3,Belgium,58381,9467,5037,817,11589616,376,42,42659,5.6
4,Brazil,514849,29314,2422,138,212559409,25,34,14103,2.2
